In [1]:
% matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import time
from time import strftime
from datetime import datetime
import matplotlib 
from IPython.core.display import clear_output
from sklearn.externals import joblib
#import nlkt

In [3]:
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize

In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Some cool words

https://blog.hubspot.com/sales/sensory-words-to-spice-up-your-sales-pitch

I am going to use this great reference for most of my text cleaning and feature extraction

https://machinelearningmastery.com/clean-text-machine-learning-python/

# Getting features from HTML

In [4]:
def get_parser(html):
    
    ### returns the parser given a raw html 
    
    x=BeautifulSoup(html.text,"html.parser")
    
    return x

In [5]:
def num_of_videos(parser,box):
    
    ### returns the number of videos for the different boxes
    ### This also checks for youtube videos  -- that are linked different in Kickstarter [NOT IN MVP]
    
    if box=="about":
        
        # Normal video
        a=len(parser.find('div',class_='full-description js-full-description responsive' + '-media formatted-lists'
        ).find_all('div', class_="video-player"))
        
        # YouTube Video
        #b=len(parser.find('div',class_='full-description js-full-description responsive' + '-media formatted-lists'
        #).find_all('div', class_="video-player"))
        
        return a
    
    elif box=="risk":
        
        return len(parser.find('div',class_='mb3 mb10-sm mb3 js-risks'
        ).find_all('div', class_="video-player"))

In [6]:
def num_of_pics(parser,box):
    
    ### returns the number of videos for the different boxes
    
    if box=="about":
        
        return len(parser.find(
            'div',
            class_='full-description js-full-description responsive-media formatted-lists'
        ).find_all('img'))
    
    elif box=="risk":
        
        return len(parser.find('div',class_='mb3 mb10-sm mb3 js-risks'
        ).find_all('img'))

In [7]:
def is_pitch_video(parser):
    
    ### returns True is the pitching pic is a video and returns false otherwise
    
    obj=parser.find(
            'div',
            class_='mx-4 mx-12-md mx0-lg'
        ).find_all('div', class_="aspect-ratio aspect-ratio--16x9 w100p ksr-video-player bg-black")
    
    if len(obj)>=1:
        return True
    
    else:
        return False

In [8]:
def num_of_perks(parser):
     
    ### returns the number of perks for a given campaing
    
    #This are the available perks
    a=len(parser.find(
            'div',
            class_="NS_projects__rewards_list js-project-rewards"
        ).find_all('li',class_="hover-group js-reward-available pledge--available pledge-selectable-sidebar"))
    
    # This are the perks that are gone -- For the MVP I will put them together but they might be 
    
    b=len(parser.find(
            'div',
            class_="NS_projects__rewards_list js-project-rewards"
        ).find_all('li',class_="hover-group pledge--all-gone pledge-selectable-sidebar"))
    
    return a+b

In [9]:
def price_in_perks(parser):
    
    ### Returns an array of the amount asked for each perk in order of appearence
    
    # For available perks
    per=parser.find(
            'div',
            class_="NS_projects__rewards_list js-project-rewards"
        ).find_all('li',class_="hover-group js-reward-available pledge--available pledge-selectable-sidebar")
    
    l=[]
    for i in range(len(per)):

        r=per[i].find_all("span",class_="money")[0].text
    
        r=r.replace(",","")
        r=r.replace(".","")
        
        l.append([int(''.join(i)) for is_digit, i in groupby(r, str.isdigit) if is_digit][0])
   
    l=np.array(l)
    
    # For unavilable perks
    
    per1=parser.find(
            'div',
            class_="NS_projects__rewards_list js-project-rewards"
        ).find_all('li',class_="hover-group pledge--all-gone pledge-selectable-sidebar")
    
    if len(per1)==1:
        return l
    else:
        l1=[]
        for i in range(len(per)):

            r=per1[i].find_all("span",class_="money")[0].text
    
            r=r.replace(",","")
            r=r.replace(".","")
        
            l1.append([int(''.join(i)) for is_digit, i in groupby(r, str.isdigit) if is_digit][0])
   
        l1=np.array(l1)
     
        return np.concatenate((l,l1))

In [10]:
def get_text(parser,box):
    
    ### Gets the text for a given box in the campaing.
    ### this will only clean basic html feature, remove bold, italic, etc
    ### it will keep the raw text
    
    if box=="about":
        
        try:
            text=parser.find(
                'div',
                class_='full-description js-full-description responsive-media formatted-lists'
            ).get_text(' ')

            text=" ".join(text.split()).strip()
            
        except AttributeError:  
            
            text='NA'
        
        return text
    
    elif box=="risk":
        
        try:
            text=parser.find(
                'div',
                class_='mb3 mb10-sm mb3 js-risks'
            ).get_text(' ')

            text=" ".join(text.split()).strip()
            
        except AttributeError:  
            
            text='NA'
        
        return text.strip("Risks and challenges").strip("Learn about accountability on Kickstarter")
    
    elif box=="perks":
        
        try:
            text=parser.find(
                'div',
                class_='NS_projects__rewards_list js-project-rewards'
            ).get_text(' ')

            text=" ".join(text.split()).strip()
            
        except AttributeError:  
            
            text='NA'
        
        return text
        

In [11]:
def num_of_sentences(text):
    if text=="NA":
        return np.nan
    else:
    
        sentences = sent_tokenize(text)

        return len(sentences)

In [117]:
np.nan

nan

In [22]:
def num_of_words(text):
    
    if text=="NA":
        return np.nan
    else:
    
        tokens = word_tokenize(text)

        #this removes all characters that are not words
        words = [word for word in tokens if word.isalpha()]

        return len(words)

In [21]:
def num_of_links(parser,box):
    
    if box == 'about':
        try:
            return len(parser.find(
                'div',
                class_='full-description js-full-description responsive-media formatted-lists'
            ).find_all('a'))
        except AttributeError:  
            
            return np.nan
            
           
    elif box == 'risk':
        try:
            return len(parser.find(
                'div',
                class_='mb3 mb10-sm mb3 js-risks'
            ).find_all('a'))-1
        except AttributeError:  
            
            return np.nan

In [13]:
meta=pd.read_csv("/kickstarter_meta.csv",index_col="index1")

In [14]:
seed = np.random.seed(42)
meta_sample = meta.sample(50000)

In [15]:
meta_sample=meta_sample.reset_index()

In [16]:
meta_sample

,index1,backers_count,blurb,category,created_at,deadline,goal,name,pledged,country,state,subcategory,link,percentage_founded,founded_bool
0,71732,822,America...the late 60’s. Hippies and the count...,games,2014-02-04 15:14:53,2014-03-15 01:00:00,5000.0,Greed,26208.00,US,TX,tabletop games,https://www.kickstarter.com/projects/101637482...,524.160000,True
1,125268,7,Join me to learn the Art of Composite molding ...,technology,2014-08-11 00:36:09,2014-09-15 01:48:22,25000.0,Composite Molding and fabrication for a 1/4 TA...,625.00,US,FL,flight,https://www.kickstarter.com/projects/148480575...,2.500000,False
2,48219,708,The Shadow Cabal is a high-impact Action/Adven...,film & video,2012-10-06 22:29:46,2012-12-11 13:57:31,20000.0,The Shadow Cabal (feature film),33327.00,US,UT,narrative film,https://www.kickstarter.com/projects/arrowstor...,166.635000,True
3,120597,114,YouMatter is an app that allows you to send pr...,technology,2015-07-08 19:08:07,2015-09-02 07:25:29,15000.0,You Matter App,15164.27,US,PA,apps,https://www.kickstarter.com/projects/140852219...,101.095133,True
4,88850,124,"David Dalla G's second album ""All Sides"" is co...",music,2015-01-26 13:40:30,2015-03-03 01:59:00,5000.0,All Sides: David Dalla G's Second Album,10025.00,US,MT,hip-hop,https://www.kickstarter.com/projects/105391897...,200.500000,True
5,33555,3,Tie Buddy. Simple solution to replace the tie ...,fashion,2015-02-02 20:44:32,2015-03-11 11:14:17,3500.0,The Tie Buddy,13.00,US,AR,accessories,https://www.kickstarter.com/projects/122532390...,0.371429,False
6,90176,7,"My goal is to produce10-13 tracks album, mix a...",music,2016-02-18 01:24:41,2016-04-29 21:39:01,6600.0,Support me bring hope to the hopeless with my ...,365.00,US,TX,faith,https://www.kickstarter.com/projects/194461040...,5.530303,False
7,129001,16,This collage of found texts confronts the scar...,theater,2011-06-07 22:31:46,2011-07-01 23:02:00,1250.0,Laer's Last Prayer,2155.00,US,IL,NaN,https://www.kickstarter.com/projects/130610441...,172.400000,True
8,57715,32,New food and cooking magazine serving Northwes...,food,2012-08-14 19:05:52,2012-10-04 15:03:55,2000.0,Spoon Magazine,3553.00,US,IL,NaN,https://www.kickstarter.com/projects/268715856...,177.650000,True
9,73218,88,56 original pen & ink drawings inspired by the...,games,2013-02-26 13:22:10,2014-09-24 16:16:27,4000.0,Renaissance Playing Cards - each unique card h...,4089.00,US,MN,playing cards,https://www.kickstarter.com/projects/424368844...,102.225000,True


In [17]:
new_features=["index1","num_videos","num_pics",
              "num_of_sent_about","num_of_sent_risk",
             "num_of_words_about","num_of_words_risk","num_links_about","num_of_links_risk"]
df_features=pd.DataFrame(columns=new_features)

In [155]:
df_features

,index1,num_videos,num_pics,num_of_sent_about,num_of_sent_risk,num_of_words_about,num_of_words_risk,num_links_about,num_of_links_risk


In [19]:
filename = '/Scraped_HTML/scraped_html_33000-33999.pkl'
scraped = joblib.load(filename)


In [23]:
for index, row in scraped.iterrows():
    
    par=get_parser(row["HTML"])
    
    text_about=get_text(par,"about")
    text_risk=get_text(par,"risk")
    
    
    
    features=(row["index1"],num_of_videos(par,"about"),num_of_pics(par,"about"),num_of_sentences(text_about),
              num_of_sentences(text_risk),num_of_words(text_about),
             num_of_words(text_risk),num_of_links(par,"about"),num_of_links(par,"risk"))
    
    
    #print(features)
    df_features.loc[index]=features